# /workspace/pluto/notebook README

このREADMEは **READMEだけで理解できること** を目標に、以下の3つを
**ユーザーが実際に可視化・実行できる粒度** でまとめたチュートリアルです。

1. **nuPlan datasetのデータ構造と読み取り方（可視化まで）**  
2. **nuPlan devkitの使い方（何をどうすると何ができるか）**  
3. **Bostonデータを使った学習手順（NaN回避〜GPU学習まで）**

---

## 1. nuPlan datasetのデータ構造と読み取り方（可視化まで）

### 1.1 物理構成（ユーザーが迷わないための最小定義）
- **Log**: `.db` 1ファイル（SQLite）  
- **Scenario**: Logの中の連続した時間区間  
- **Frame / Iteration**: シナリオ内の1タイムステップ（Lidar 1フレーム）

### 1.2 必須の環境変数
nuPlan devkitは以下を前提に動きます。
- `NUPLAN_DATA_ROOT` : `.db` と `splits` の起点
- `NUPLAN_MAPS_ROOT` : mapデータのルート
- `NUPLAN_MAP_VERSION` : mapバージョン

### 1.3 どこに何があるか（Bostonの例）
```
/nuplan/dataset/nuplan-v1.1/
  splits/
    train_boston/
      *.db   ← この数がログ数（シナリオ数ではない）
```

**シナリオ数を把握したい場合**は、まず `.db` を数えるのが正しい出発点です。  
（ディレクトリ数は0で誤りになる）

### 1.4 `.db` を読み、シナリオを1つ取り出す
以下は「1つのシナリオを取得して中身を確認する」最小コードです。

```python
from nuplan.planning.scenario_builder.nuplan_db.nuplan_scenario_builder import NuPlanScenarioBuilder
from nuplan.planning.scenario_builder.scenario_filter import ScenarioFilter
from nuplan.planning.utils.multithreading.worker_sequential import Sequential

# 1) ScenarioBuilder を作る（環境変数が必要）
scenario_builder = NuPlanScenarioBuilder(
    data_root=os.environ["NUPLAN_DATA_ROOT"],
    map_root=os.environ["NUPLAN_MAPS_ROOT"],
    map_version=os.environ["NUPLAN_MAP_VERSION"],
)

# 2) ScenarioFilter を作る（最小で1件だけ）
scenario_filter = ScenarioFilter(
    log_names=None,
    map_names=None,
    scenario_tokens=None,
    num_scenarios_per_type=1,
    limit_total_scenarios=1,
    timestamp_threshold_s=0.0,
    ego_displacement_minimum_m=0.0,
    ego_start_speed_threshold=0.0,
    ego_stop_speed_threshold=0.0,
    expand_scenarios=True,
    remove_invalid_goals=True,
    shuffle=True,
)

worker = Sequential()
scenarios = scenario_builder.get_scenarios(scenario_filter, worker)
scenario = scenarios[0]
```

### 1.5 シナリオから「可視化に必要な最低限の情報」を取る
**ユーザーが可視化できるレベルの情報**は以下です。

```python
# フレーム数
num_frames = scenario.get_number_of_iterations()

# Lidarフレームのトークン列（可視化に使う）
tokens = scenario.get_scenario_tokens()

# Egoの軌跡（位置・向き）
ego_xy = []
for i in range(num_frames):
    ego_state = scenario.get_ego_state_at_iteration(i)
    ego_xy.append((ego_state.rear_axle.x, ego_state.rear_axle.y))
```

### 1.6 Ego軌跡を簡単に可視化する
```python
import matplotlib.pyplot as plt

xs = [p[0] for p in ego_xy]
ys = [p[1] for p in ego_xy]

plt.figure(figsize=(6,6))
plt.plot(xs, ys, linewidth=2)
plt.axis("equal")
plt.title("Ego Trajectory")
plt.show()
```

### 1.7 データ解像度の注意点（8点 vs 80ステップ）
- **8点の軌跡**は「可視化用の粗サンプル」。  
- 学習・推論は **0.1秒刻みの高解像度**（未来 8秒 = 80ステップ）を前提。  
  - **入力**: 過去 2秒 → 21ステップ  
  - **出力**: 未来 8秒 → 80ステップ  
- **8点軌跡 = 学習用の正規データではない** と理解することが重要。

---

## 2. nuPlan devkitの使い方（何をどうしたら何ができるか）

### 2.1 devkitでできること（具体例）
1. `.db` からシナリオを抽出  
2. ScenarioFilter で「特定条件のシナリオだけ」を抽出  
3. シナリオのフレームを可視化（map + Lidar）  
4. GIFとしてアニメーション出力  
5. Egoの速度・位置・軌跡などを時系列で可視化

### 2.2 何をどうすると「シナリオ抽出」できるか
**やること**: `ScenarioFilter` を作り、`scenario_builder.get_scenarios()` で抽出する。

```python
scenario_filter = ScenarioFilter(
    log_names=["2021.08.12.12.35.09_veh-28_02678_02925"],  # 例: 特定ログのみ
    map_names=None,
    scenario_tokens=None,
    num_scenarios_per_type=1,
    limit_total_scenarios=1,
    timestamp_threshold_s=0.0,
    ego_displacement_minimum_m=2.0,  # 2m以上動いたシナリオのみ
    ego_start_speed_threshold=0.0,
    ego_stop_speed_threshold=0.0,
    expand_scenarios=True,
    remove_invalid_goals=True,
    shuffle=True,
)
```

### 2.3 何をどうすると「map + Lidar を描画」できるか
**やること**: `LidarPc.render` を呼ぶ。  
（内部で `render_on_map` が呼ばれ、map + 点群 + アノテーションが描画される）

```python
from nuplan.database.nuplan_db_orm.lidar_pc import LidarPc

token = tokens[0]  # 先頭フレームのLidarトークン
lidar = LidarPc(token, scenario._db)  # scenario._db は内部で持つDB
fig = lidar.render()
```

### 2.4 何をどうすると「GIFアニメーション」が作れるか
**やること**: 各フレームを `LidarPc.render` で描き、`FuncAnimation` でGIF化する。

```python
import matplotlib.pyplot as plt
from matplotlib import animation

fig = plt.figure(figsize=(6,6))

def update(i):
    token = tokens[i]
    lidar = LidarPc(token, scenario._db)
    fig.clear()
    lidar.render(ax=plt.gca())

ani = animation.FuncAnimation(fig, update, frames=len(tokens))
ani.save("/workspace/pluto/notebook/outputs/scenario.gif", writer="pillow")
```

---

## 3. Bostonデータを使った学習手順（NaN回避〜GPU学習まで）

### 3.1 重要な前提: TrajectorySamplingの不整合を修正
**問題**: `future_steps=80` なのに `TrajectorySampling(num_poses=8, interval_length=1)`  
→ **shape mismatch (8,3) vs (80,3)** で NaN が発生。

**修正（必須）**:
```python
# /workspace/pluto/src/models/pluto/pluto_model.py
trajectory_sampling = TrajectorySampling(
    num_poses=80,
    time_horizon=8,
    interval_length=0.1,
)
```

この修正により **CPU学習でNaNが消えることを確認済み**。

### 3.2 notebook環境での学習実行方針
- **subprocess.run() は使わない**  
  - Hydra設定が反映されず誤解を生むため。
- **Python APIで実行する**  
  - `initialize_config_dir` + `compose` + `run_training.main(cfg)`

最小例:
```python
from hydra import initialize_config_dir, compose
from nuplan.planning.script.run_training import main as training_main

config_dir = "/workspace/pluto/config"
overrides = [
    "scenario_filter=training_scenarios_boston",
    "splitter=ratio_splitter",
    "scenario_builder=nuplan_boston",
    "model=pluto_model",
]

with initialize_config_dir(config_dir=config_dir, version_base="1.2"):
    cfg = compose(config_name="default_training", overrides=overrides)
    training_main(cfg)
```

### 3.3 notebookで必要なDDP設定
- `ddp_find_unused_parameters_false` は **notebookでは非対応**。  
- notebook環境では `ddp_notebook` を使う。

```python
overrides += ["lightning.trainer.params.strategy=ddp_notebook"]
```

### 3.4 小規模でパイプライン確認
目的: データローダ・モデル・学習ループが動くか確認。  
方法: まず少量バッチで動作確認。

```python
overrides += [
    "lightning.trainer.params.max_epochs=1",
    "lightning.trainer.params.limit_train_batches=0.01",
    "lightning.trainer.params.limit_val_batches=0.01",
]
```

### 3.5 キャッシュ構造の整合性を確認する
- `pluto_model` は **`feature.gz`** を期待。  
- `simple_vector_model` は **`agents.gz` / `vector_map.gz` / `trajectory.gz`** 構造。  
- モデルとキャッシュ構造が一致しないと学習が失敗する。

### 3.6 分割キャッシュ戦略（大規模学習の基本）
- 1647シナリオを一括キャッシュすると重すぎる。  
- **分割（例: 400/400/400/447）でキャッシュ作成 → 学習** が妥当。  
- `scenario_filter` で部分集合を指定できるため、分割キャッシュは実装上問題ない。

### 3.7 1647シナリオ全データでの学習コマンド（GPU）
**キャッシュ作成**
```bash
python run_training.py \
  py_func=cache \
  +training=train_boston \
  cache.cache_path=/nuplan/exp/boston_cache_1647_full \
  cache.force_feature_computation=True
```

**学習実行**
```bash
python run_training.py \
  py_func=train \
  +training=train_boston \
  cache.cache_path=/nuplan/exp/boston_cache_1647_full \
  lightning.trainer.params.max_epochs=100 \
  lightning.trainer.params.accelerator=gpu \
  lightning.trainer.params.devices=1
```

### 3.8 失敗パターン（回避事項）
1. **subprocess.run() を使う**  
   - Hydraの新規YAMLが認識されず、誤解を招くエラーが出る。
2. **モデルとキャッシュ構造の不整合**  
   - `pluto_model` 用キャッシュは `feature.gz` を含む必要がある。
3. **8点軌跡を学習用に使えると誤解する**  
   - 学習は 80ステップ前提。



---

## W&B ログ管理のメカニズム解説

### 現在の設定

```yaml
# default_training.yaml
wandb:
  mode: disable              ← 現在は W&B 無効
  project: nuplan-pluto
  name: ${experiment_name}
  log_model: all             ← 【重要】すべてのチェックポイントをアップロード
  artifact:
  run_id:
```

```yaml
# custom_lightning.yaml
trainer:
  checkpoint:
    save_top_k: 5            ← 【重要】 top 5 モデルを保存
    monitor: loss/val_loss
    mode: min
```

---

### 問題点と目指す状態

| 項目 | 現在 | 目指す状態 |
|------|------|---------|
| **W&B アップロード** | `log_model: all` (5個すべて) | `log_model: best` (best のみ) |
| **ローカル保存** | top 5 全て保存 | top 1 + last model のみ |
| **ストレージ効率** | 低い（500MB 程度） | 高い（100MB 程度） |
| **ネットワーク効率** | 低い（5ファイル） | 高い（1ファイル） |

---

### W&B ログ管理の仕組み

#### A. `log_model` パラメータの意味

```yaml
log_model: all      # すべてのチェックポイント（epoch=0, 1, 2, 3, 4）を W&B に artifact として登録
log_model: best     # validation loss が最小のチェックポイント（best）だけを登録
log_model: false    # W&B にはアップロードしない
```

**効果:**
- `all`: W&B に 5ファイル × 100MB = 500MB アップロード
- `best`: W&B に 1ファイル × 100MB = 100MB アップロード（80%削減）

---

#### B. `save_top_k` パラメータの意味

```yaml
save_top_k: 5      # validation loss でランク付けして、 top 5 をローカルに保存
               # epoch=0, 1, 2, 3, 4 などの ckpt ファイルを /workspace/pluto/checkpoints に保存

save_top_k: 1      # validation loss が最小のモデルだけローカルに保存
               # epoch=X-best.ckpt だけが /workspace/pluto/checkpoints に残される

save_top_k: -1     # すべてのチェックポイントを保存（最大値なし）
```

**効果:**
- `save_top_k: 5`: ローカルに 5ファイル保存（合計 500MB）
- `save_top_k: 1`: ローカルに 1ファイル保存（合計 100MB）

---

#### C. 組み合わせパターン

| `save_top_k` | `log_model` | W&B | ローカル | 合計ストレージ |
|-------------|----------|-----|---------|-------------|
| 5 | all | 5個 | 5個 | 1000MB（W&B+Local） |
| 5 | best | 1個 | 5個 | 600MB（100MB W&B+500MB Local） |
| 1 | all | 1個 | 1個 | 200MB |
| **1** | **best** | **1個** | **1個** | **100MB** ← ✅ 最適 |

---

### さらに洗練した設定方法

#### オプション A: Last & Best を両方保存

```yaml
# custom_lightning.yaml
trainer:
  checkpoint:
    save_top_k: 1           # best model のみ保存
    save_last: true         # + 最後のモデルも保存（これを `last.ckpt` として）
    monitor: loss/val_loss
    mode: min
```

**結果:**
- ローカル: `best.ckpt` + `last.ckpt` = 2ファイル
- W&B: `best.ckpt` のみ = 1ファイル

---

#### オプション B: 訓練再開のための Last 専用

```yaml
# custom_lightning.yaml
trainer:
  checkpoint:
    save_top_k: 1           # validation loss が最小のモデル
    save_last: true         # 最新エポックのモデル（訓練再開用）
    monitor: loss/val_loss
    mode: min
```

---

### 設定変更時の流れ

```
変更前：
  save_top_k: 5, log_model: all
  → /checkpoints: 5ファイル
  → W&B artifact: 5ファイル

↓ 設定変更

変更後：
  save_top_k: 1, log_model: best
  → /checkpoints: 1ファイル（best）
  → W&B artifact: 1ファイル（best）
  → 次回訓練から適用
```

---

### ファイル修正前にチェックリスト

修正する前に、以下を確認してください：

```
□ 1. Best model のみ W&B にアップロード
     → default_training.yaml: log_model を all → best

□ 2. ローカル保存は 1ファイル のみ
     → custom_lightning.yaml: save_top_k を 5 → 1

□ 3. 訓練再開のための last model も必要？
     → save_last: true を追加するか検討

□ 4. .gitignore で checkpoints/ を除外？
     → git に .ckpt ファイルをコミットしない
```

---

## ストレージ保存場所の管理方針

### 4.1 実際の保存場所と使用サイズ

**調査結果（2026年1月28日）:**

| ディレクトリ | サイズ | 用途 | 問題度 |
|-----------|------|------|------|
| `/root/nuplan/exp/exp/wandb/` | 8.0M | W&B ログ | 🟢 問題なし |
| `/workspace/pluto/checkpoints/` | 97M | ローカル モデル | 🟢 管理可能 |
| `/workspace/pluto/outputs/` | 2.9M | Hydra 設定出力 | 🟡 古い実行残骸 |
| `/workspace/pluto/notebook/` | 46M | ノートブック キャッシュ | 🟡 削除可能 |
| `/workspace/pluto/exp/` | 4.0G | 歴史的実験 | 🔴 蓄積問題 |

---

### 4.2 重要な誤解の修正

**「/root が扱いにくい」という懸念について:**

```
❌ 誤解: "/root に大量のログが溜まっている"
✅ 実態: /root/nuplan/exp/exp/wandb/ は 8.0M のみ
         - 最新実行（run-20260128_223635-nd88goyf）: 8.0M
         - 旧実行（run-20260128_222233-nr111k36）: 96K
         - 合計: 8.1M → 扱いにくい原因ではない
```

**実際の問題:**

```
🔴 本当の問題: /workspace/pluto/exp/ ディレクトリが 4.0G 蓄積

理由：
  ├─ 訓練実行するたびに exp/training/{exp_name}/{timestamp}/ が自動作成される
  ├─ 過去の実験をすべて保持している（削除戦略がない）
  ├─ チェックポイントが散在している
  │  ├─ /workspace/pluto/checkpoints/（手動保存：2ファイル）
  │  ├─ /workspace/pluto/outputs/*/checkpoints/（Hydra出力）
  │  └─ /workspace/pluto/exp/training/*/*/checkpoints/（歴史的実験）
  └─ 「どの実験を保持するか」の基準がない
```

---

### 4.3 現在のチェックポイント保存フロー

```
訓練実行
  ↓
Hydra が /workspace/pluto/outputs/YYYYMMDD_HHMMSS/ を自動作成
  ↓
Lightning が checkpoints/ に save_top_k: 5 個保存
  ↓
ユーザーが手動で /workspace/pluto/checkpoints/ にコピー
  ↓
古い出力は自動削除されず /workspace/pluto/exp/training/ に蓄積
  ↓
4.0G に膨張 ←【ここが問題】
```

---

### 4.4 W&B と ローカル保存の関係

**現在の設定:**
```yaml
# default_training.yaml
wandb:
  log_model: all              # すべてのチェックポイントをアップロード

# custom_lightning.yaml
trainer:
  checkpoint:
    save_top_k: 5            # top 5 モデルをローカルに保存
```

**結果:**
- W&B に 5 ファイルアップロード（100MB × 5 = 500MB）
- ローカルに 5 ファイル保存（100MB × 5 = 500MB）
- **合計 1000MB の重複保存** ← 最適化可能

---

### 4.5 推奨される保存戦略

#### **戦略 A: 本番用（最小化型）**

```yaml
# default_training.yaml
wandb:
  log_model: best             # ← 最良のモデルのみアップロード

# custom_lightning.yaml
trainer:
  checkpoint:
    save_top_k: 1            # ← 最良のモデルのみローカルに保存
    save_last: true          # ← 訓練再開用に最終モデルも保存（オプション）
    monitor: loss/val_loss
    mode: min
```

**結果:**
- W&B: 1 ファイル（100MB）
- ローカル: 2 ファイル（best + last = 200MB）
- **合計 300MB（80% 削減）** ✅

---

#### **戦略 B: 実験用（柔軟型）**

```yaml
# default_training.yaml
wandb:
  log_model: false            # ← W&B にはアップロードしない（ネットワーク効率）

# custom_lightning.yaml
trainer:
  checkpoint:
    save_top_k: 3            # ← top 3 をローカルに保存（実験段階の確認用）
    monitor: loss/val_loss
    mode: min
```

**利点:**
- ネットワーク帯域幅の節約
- ローカルでいくつかの候補を比較可能
- W&B は訓練ログのみ記録

---

### 4.6 ディレクトリ一元化戦略

**目標: チェックポイントを一箇所に集約**

```
改善前（散在）:
├─ /workspace/pluto/checkpoints/
│  ├─ botson_1K_25_epochs.ckpt
│  └─ pluto_1M_aux_cil.ckpt
├─ /workspace/pluto/outputs/boston_100epoch/checkpoints/
└─ /workspace/pluto/exp/training/*/*/checkpoints/

改善後（一元化）:
└─ /workspace/pluto/checkpoints/
   ├─ boston_1K_25ep_20260128/
   │  ├─ best.ckpt
   │  └─ last.ckpt
   ├─ mini_cache_2ep_20260127/
   │  ├─ best.ckpt
   │  └─ last.ckpt
   └── [訓練実験ごとのディレクトリ]
```

**実装方法:**
1. Hydra の `run.dir` を `/workspace/pluto/checkpoints/{experiment_name}/` に変更
2. `save_top_k` と `save_last` で必要なモデルのみ保存
3. 古い実験は `/workspace/pluto/archive/` に月ごとに移動

---

### 4.7 Hydra 出力のリダイレクト（オプション）

**問題:** Hydra が出力する config/hydra ファイルが `/workspace/pluto/outputs/` に蓄積

**解決策 A: 自動削除**
```yaml
# 訓練後に自動削除スクリプト
import shutil
shutil.rmtree(cfg.hydra_output_subdir)
```

**解決策 B: 別ディレクトリに集約**
```yaml
hydra:
  output_subdir: .hydra_logs/${now:%Y%m%d_%H%M%S}
```


---

### 参考: 各ストレージの役割まとめ

| 場所 | 役割 | 管理方法 |
|------|------|--------|
| `/workspace/pluto/checkpoints/` | 本番・保存済みモデル | 手動で精選 |
| `/workspace/pluto/outputs/` | 現在の訓練出力 | Hydra が自動生成 |
| `/workspace/pluto/exp/` | 過去の実験履歴 | 月ごとにアーカイブ推奨 |
| `/root/nuplan/exp/exp/wandb/` | W&B ログ | W&B が管理（8.0M） |
| `W&B Cloud` | 実験ログ・アーティファクト | `log_model` で制御 |

## 推論＋アニメーション化: map + ego + others

### 5.1 概要

保存済みの PLUTO モデルを使用して、nuPlan シナリオに対して推論を行い、以下の3要素をアニメーション化します：

- **Map** 🗺️: シナリオ内の道路・レーン情報
- **Ego** 🚗: 自車両の過去軌跡と予測軌跡
- **Others** 🚙: 周辺車両の過去軌跡と予測軌跡

### 5.2 必要なライブラリ

In [1]:
import os
import pickle
import torch
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation, PillowWriter
from matplotlib.patches import Rectangle, FancyBboxPatch
from matplotlib.collections import PatchCollection

# nuPlan
from nuplan.planning.scenario_builder.nuplan_db.nuplan_scenario_builder import NuPlanScenarioBuilder
from nuplan.planning.scenario_builder.scenario_filter import ScenarioFilter
from nuplan.planning.utils.multithreading.worker_sequential import Sequential

# PLUTO
import sys
sys.path.insert(0, '/workspace/pluto')
from src.models.pluto.pluto_model import PlanningModel
from src.feature_builders.pluto_feature_builder import PlutoFeatureBuilder


/opt/conda/envs/pluto/lib/python3.9/site-packages/wandb/apis/public.py:3109: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import parse_version
/opt/conda/envs/pluto/lib/python3.9/site-packages/timm/models/layers/__init__.py:49: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)


### 5.3 モデル読み込み

In [7]:
checkpoint_path = "/workspace/pluto/checkpoints/botson_1K_25_epochs.ckpt"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# チェックポイント読み込み
checkpoint = torch.load(checkpoint_path, map_location=device)

# state_dict を抽出
if 'state_dict' in checkpoint:
    state_dict = checkpoint['state_dict']
else:
    state_dict = checkpoint

# ✅ キーの "model." プレフィックスを削除
fixed_state_dict = {}
for key, value in state_dict.items():
    if key.startswith('model.'):
        new_key = key[6:]  # "model." を削除（6文字）
        fixed_state_dict[new_key] = value
    else:
        fixed_state_dict[key] = value

# ✅ モデルをインスタンス化（パラメータ指定）
model = PlanningModel(
    dim=128,
    state_channel=6,
    polygon_channel=6,
    history_channel=9,
    history_steps=21,
    future_steps=80,
    encoder_depth=4,
    decoder_depth=4,
    num_heads=4,
    num_modes=12,
)

# ✅ 重みをロード
model.load_state_dict(fixed_state_dict)
model = model.to(device)
model.eval()

print("✅ Model loaded successfully!")

✅ Model loaded successfully!


### 5.4 シナリオデータの読み込み

In [13]:
import glob

# 環境変数を確認
os.environ['NUPLAN_DATA_ROOT'] = '/nuplan/dataset/nuplan-v1.1'
os.environ['NUPLAN_MAPS_ROOT'] = '/nuplan/dataset/nuplan-v1.1/maps'
os.environ['NUPLAN_MAP_VERSION'] = 'nuplan-maps-v0.1'

# ✅ sensor_root を追加
sensor_root = os.path.join(os.environ['NUPLAN_DATA_ROOT'], 'sensor_blobs')

# ✅ db_files を取得（Boston 訓練データ）
db_files = sorted(glob.glob(os.path.join(
    os.environ['NUPLAN_DATA_ROOT'],
    'splits/train_boston/*.db'
)))

print(f"Found {len(db_files)} DB files in Boston train set")
print(f"First DB: {db_files[0]}")

# ScenarioBuilder を作成
scenario_builder = NuPlanScenarioBuilder(
    data_root=os.environ['NUPLAN_DATA_ROOT'],
    map_root=os.environ['NUPLAN_MAPS_ROOT'],
    map_version=os.environ['NUPLAN_MAP_VERSION'],
    sensor_root=sensor_root,  # ✅ 新しい引数
    db_files=db_files,         # ✅ 新しい引数
)


Found 1647 DB files in Boston train set
First DB: /nuplan/dataset/nuplan-v1.1/splits/train_boston/2021.08.18.18.32.06_veh-28_00049_00111.db


In [ ]:
# 条件をゆるくして、有効なシナリオを探す
scenario_filter = ScenarioFilter(
    scenario_types=None,
    scenario_tokens=None,
    log_names=None,
    map_names=None,
    num_scenarios_per_type=1,
    limit_total_scenarios=1,
    timestamp_threshold_s=0.0,
    ego_displacement_minimum_m=1.0,  # 1m以上移動
    expand_scenarios=False,           # 展開しない
    remove_invalid_goals=True,
    shuffle=False,
)

# シナリオを取得
scenarios = scenario_builder.get_scenarios(scenario_filter, Sequential())
scenario = scenarios[0]
print(f"Scenario: {scenario.scenario_name}")
print(f"Duration (sec): {scenario.duration_s.time_s:.2f}")
tokens = scenario.get_scenario_tokens()
print(f"Token count: {len(tokens)}")

Scenario: 00bf712a8e9e59bf
Duration (sec): 20.00
Token count: 401



### 5.5 特徴抽出と推論


In [21]:
# 実際のシナリオデータから推論を実行する場合
# （PlutoFeatureBuilder が必要）

from nuplan.planning.feature_builder.builders.pluto_feature_builder import PlutoFeatureBuilder

# Feature Builder インスタンス化
feature_builder = PlutoFeatureBuilder()

# シナリオからトークンを取得
tokens = scenario.get_scenario_tokens()

# 各フレームで推論
predictions = []
for token_idx, token in enumerate(tokens[:10]):  # 最初の10フレームのみ
    try:
        # 実際の特徴抽出（複雑な処理）
        # このステップは PlutoFeatureBuilder 内部で実装される
        # ここでは概略のみ示す
        
        # 簡易版: スキップして次へ
        pass
    except Exception as e:
        print(f"Frame {token_idx}: {e}")
        continue

print("✅ Inference completed")

ModuleNotFoundError: No module named 'nuplan.planning.feature_builder'

In [20]:
# パラメータ定義
history_steps = 21
future_steps = 80


# 2) キャッシュ（feature.gz）を使う方法
# または生データから直接特徴抽出
cache_path = "/nuplan/exp/boston_cache_1647_full"

# 3) バッチを構築して推論
with torch.no_grad():
    # feature_builder.build_features() でテンソル化
    # ここでは擬似的に入力テンソルを構築
    batch = {
        'ego_history': torch.randn(1, history_steps, 6, device=device),
        'agents': torch.randn(1, 50, history_steps, 6, device=device),  # 最大 50 agent
        'map_lanes': torch.randn(1, 500, 6, device=device),
        'map_polygons': torch.randn(1, 100, 6, device=device),
    }
    
    # 推論実行
    output = model(batch)
    # output は multimodal predictions (modes, future_steps, 2)
    trajectories = output['trajectories']  # (batch, modes, future_steps, 2)

KeyError: 'agent'


### 5.6 ビジュアライゼーション（静止画）



```python
def plot_scenario_snapshot(scenario, token_idx, ego_trajectory, agent_trajectories, save_path=None):
    """
    1フレームの状態をプロット
    
    Args:
        scenario: nuPlan scenario
        token_idx: フレームインデックス
        ego_trajectory: shape (future_steps, 2) - 予測軌跡
        agent_trajectories: dict {agent_id: (future_steps, 2)}
        save_path: 保存先（None の場合は表示のみ）
    """
    fig, ax = plt.subplots(figsize=(12, 12))
    
    # 1) Map を描画
    token = scenario._db.lidar_pc[token_idx].token
    from nuplan.planning.utils.rendering.render_radius_utils import render_radius_utils
    
    ego_state = scenario._db.ego_pv(token)
    ego_pos = np.array([ego_state.x, ego_state.y])
    
    # Map レーンを描画（簡略版）
    ax.set_xlim(ego_pos[0] - 100, ego_pos[0] + 100)
    ax.set_ylim(ego_pos[1] - 100, ego_pos[1] + 100)
    ax.set_aspect('equal')
    
    # 2) Ego 軌跡を描画
    # 過去軌跡（黒）
    ego_history = np.array([
        np.array([scenario._db.ego_pv(scenario._db.lidar_pc[token_idx - i].token).x,
                  scenario._db.ego_pv(scenario._db.lidar_pc[token_idx - i].token).y])
        for i in range(min(21, token_idx))
    ][::-1])
    
    ax.plot(ego_history[:, 0], ego_history[:, 1], 'k-', linewidth=2, label='Ego History')
    ax.plot(ego_pos[0], ego_pos[1], 'go', markersize=10, label='Ego Current')
    
    # 予測軌跡（赤）
    if ego_trajectory is not None:
        predicted_trajectory = ego_pos[np.newaxis, :] + ego_trajectory
        ax.plot(predicted_trajectory[:, 0], predicted_trajectory[:, 1], 
                'r-', linewidth=2, label='Ego Prediction', alpha=0.7)
        ax.plot(predicted_trajectory[-1, 0], predicted_trajectory[-1, 1], 
                'r*', markersize=15, label='Ego End')
    
    # 3) Other agents
    agents = scenario.get_agents_at(token)
    for agent_id, agent_state in agents.items():
        agent_pos = np.array([agent_state.x, agent_state.y])
        
        # 過去軌跡（グレー）
        ax.plot(agent_pos[0], agent_pos[1], 'bs', markersize=8, alpha=0.5)
        
        # 予測軌跡（青）
        if agent_id in agent_trajectories:
            pred_traj = agent_pos[np.newaxis, :] + agent_trajectories[agent_id]
            ax.plot(pred_traj[:, 0], pred_traj[:, 1], 'b-', linewidth=1.5, alpha=0.5)
    
    ax.legend(loc='upper right')
    ax.set_xlabel('x (m)')
    ax.set_ylabel('y (m)')
    ax.set_title(f'Scenario: {scenario.scenario_name}, Frame: {token_idx}')
    ax.grid(True, alpha=0.3)
    
    if save_path:
        plt.savefig(save_path, dpi=100, bbox_inches='tight')
    plt.show()

# 使用例
plot_scenario_snapshot(
    scenario, 
    token_idx=30,
    ego_trajectory=trajectories[0, 0].cpu().numpy(),  # 最初のモード
    agent_trajectories={},
    save_path='/workspace/pluto/notebook/outputs/frame_0.png'
)
```



---

### 5.7 アニメーション化



```python
def create_animation(scenario, ego_trajectories_list, agent_trajectories_list, 
                     output_path, fps=10, duration_frames=100):
    """
    複数フレームをアニメーション化
    
    Args:
        scenario: nuPlan scenario
        ego_trajectories_list: list of (future_steps, 2) arrays
        agent_trajectories_list: list of dicts {agent_id: (future_steps, 2)}
        output_path: GIF 出力先
        fps: フレームレート
        duration_frames: アニメーションのフレーム数
    """
    fig, ax = plt.subplots(figsize=(14, 14))
    
    def update(frame_idx):
        ax.clear()
        
        # フレームを取得
        token = scenario._db.lidar_pc[min(frame_idx, len(scenario._db.lidar_pc) - 1)].token
        ego_state = scenario._db.ego_pv(token)
        ego_pos = np.array([ego_state.x, ego_state.y])
        
        # 表示範囲
        ax.set_xlim(ego_pos[0] - 80, ego_pos[0] + 80)
        ax.set_ylim(ego_pos[1] - 80, ego_pos[1] + 80)
        ax.set_aspect('equal')
        
        # 1) Ego 現在位置
        ax.plot(ego_pos[0], ego_pos[1], 'go', markersize=15, label='Ego', zorder=5)
        
        # 2) Ego 過去軌跡
        if frame_idx > 0:
            ego_history = []
            for i in range(min(21, frame_idx)):
                hist_state = scenario._db.ego_pv(scenario._db.lidar_pc[frame_idx - i].token)
                ego_history.append([hist_state.x, hist_state.y])
            ego_history = np.array(ego_history[::-1])
            ax.plot(ego_history[:, 0], ego_history[:, 1], 'g--', linewidth=1.5, alpha=0.5)
        
        # 3) Ego 予測軌跡（複数モード）
        if frame_idx < len(ego_trajectories_list):
            for mode_idx, traj in enumerate(ego_trajectories_list[frame_idx]):
                pred_traj = ego_pos[np.newaxis, :] + traj
                alpha = 0.7 if mode_idx == 0 else 0.3
                linewidth = 2 if mode_idx == 0 else 1
                ax.plot(pred_traj[:, 0], pred_traj[:, 1], 'r-', 
                       linewidth=linewidth, alpha=alpha, label='Ego Pred' if mode_idx == 0 else '')
        
        # 4) Other agents
        agents = scenario.get_agents_at(token)
        for agent_id, agent_state in agents.items():
            agent_pos = np.array([agent_state.x, agent_state.y])
            ax.plot(agent_pos[0], agent_pos[1], 'bs', markersize=10, alpha=0.7)
            
            # Agent 予測軌跡
            if frame_idx < len(agent_trajectories_list):
                if agent_id in agent_trajectories_list[frame_idx]:
                    pred_traj = agent_pos[np.newaxis, :] + agent_trajectories_list[frame_idx][agent_id]
                    ax.plot(pred_traj[:, 0], pred_traj[:, 1], 'b-', linewidth=1, alpha=0.5)
        
        ax.legend(loc='upper right', fontsize=10)
        ax.set_xlabel('x (m)')
        ax.set_ylabel('y (m)')
        ax.set_title(f'{scenario.scenario_name} | Frame: {frame_idx}/{duration_frames}')
        ax.grid(True, alpha=0.2)
    
    # アニメーション作成
    ani = FuncAnimation(fig, update, frames=duration_frames, interval=1000//fps)
    
    # GIF に保存
    writer = PillowWriter(fps=fps)
    ani.save(output_path, writer=writer)
    print(f"✅ Animation saved to {output_path}")
    plt.close(fig)



# 使用例
# 前処理: 複数フレームに対して推論を実行
ego_trajectories_list = []
agent_trajectories_list = []

for frame_idx in range(50):
    # 推論実行（前の section 参照）
    ego_traj = trajectories[0, :, :].cpu().numpy()  # (modes, future_steps, 2)
    
    ego_trajectories_list.append(ego_traj)
    agent_trajectories_list.append({})

create_animation(
    scenario,
    ego_trajectories_list,
    agent_trajectories_list,
    output_path='/workspace/pluto/notebook/outputs/scenario_animation.gif',
    fps=10,
    duration_frames=50
)
```

---



### 5.8 実装上のポイント

#### A. キャッシュデータの活用



```python
# feature.gz を直接読み込む方法
import gzip
import pickle

cache_dir = "/nuplan/exp/boston_cache_1647_full"
feature_file = os.path.join(cache_dir, "feature.gz")

with gzip.open(feature_file, 'rb') as f:
    cached_features = pickle.load(f)
    # features には feature_data_list が含まれる
```



#### B. Multimodal Prediction の可視化



```python
# 複数の予測モード（候補軌跡）を表示
colors = plt.cm.Spectral(np.linspace(0, 1, num_modes))

for mode_idx in range(num_modes):
    trajectory = trajectories[0, mode_idx, :, :].cpu().numpy()
    predicted = ego_pos[np.newaxis, :] + trajectory
    
    ax.plot(predicted[:, 0], predicted[:, 1], 
            color=colors[mode_idx], linewidth=1.5, alpha=0.5)
```



#### C. Agent の予測軌跡



```python
# Agent Predictor から他車両の軌跡も取得可能
# model.agent_predictor() を呼び出して agent の multimodal 予測を生成
```

---



### 5.9 全体フロー

```
1️⃣ チェックポイント読み込み
   ↓
2️⃣ シナリオデータ読み込み
   ↓
3️⃣ 各フレームで推論実行
   - Ego 軌跡（multimodal）
   - Agent 軌跡（multimodal）
   ↓
4️⃣ ビジュアライゼーション
   - 静止画（snapshot）
   - アニメーション（GIF）
   ↓
5️⃣ 出力保存
   /workspace/pluto/notebook/outputs/ に保存
```

---

### 5.10 実行コード一式

**全体を実行するノートブックセルの例:**



```python
# 全セットアップ
import os
import pickle
import torch
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation, PillowWriter

# パス設定
os.environ['NUPLAN_DATA_ROOT'] = '/nuplan/dataset/nuplan-v1.1'
os.environ['NUPLAN_MAPS_ROOT'] = '/nuplan/dataset/nuplan-v1.1/maps'
os.environ['NUPLAN_MAP_VERSION'] = 'nuplan-maps-v0.1'

# 1. モデル読み込み
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
checkpoint_path = "/workspace/pluto/checkpoints/botson_1K_25_epochs.ckpt"

from src.models.pluto.pluto_model import PlanningModel
model = PlanningModel()
checkpoint = torch.load(checkpoint_path, map_location=device)
if 'state_dict' in checkpoint:
    model.load_state_dict(checkpoint['state_dict'])
model = model.to(device)
model.eval()

# 2. シナリオ読み込み
from nuplan.planning.scenario_builder.nuplan_db.nuplan_scenario_builder import NuPlanScenarioBuilder
from nuplan.planning.scenario_builder.scenario_filter import ScenarioFilter
from nuplan.planning.utils.multithreading.worker_sequential import Sequential

scenario_builder = NuPlanScenarioBuilder(
    data_root=os.environ['NUPLAN_DATA_ROOT'],
    map_root=os.environ['NUPLAN_MAPS_ROOT'],
    map_version=os.environ['NUPLAN_MAP_VERSION'],
)

scenario_filter = ScenarioFilter(scenario_types=['starting_high_speed_turn'], num_scenarios=1)
scenarios = scenario_builder.get_scenarios(scenario_filter, Sequential())
scenario = scenarios[0]

# 3. 推論 → アニメーション化
create_animation(scenario, [], [], '/workspace/pluto/notebook/outputs/result.gif')
```

---



### 参考: nuPlan Rendering Utils

nuPlan には標準の描画ツールがあります：



```python
from nuplan.planning.utils.rendering.rendering_engine import RenderingEngine
from nuplan.planning.utils.rendering.rendering_types import TrajectoryType

engine = RenderingEngine()
# ビルトインの render 関数で Map/Ego/Others を描画
```